In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+ '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)

In [6]:
ctable.indices_char

{0: ' ',
 1: '+',
 2: '0',
 3: '1',
 4: '2',
 5: '3',
 6: '4',
 7: '5',
 8: '6',
 9: '7',
 10: '8',
 11: '9'}

# Data Generation

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 80000


In [8]:
print(questions[:5], expected[:5])

['7+4    ', '42+785 ', '866+67 ', '27+4   ', '28+578 '] ['11  ', '827 ', '933 ', '31  ', '606 ']


# Processing

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(expected), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [10]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:20000]
train_y = y[:20000]
test_x = x[20000:]
test_y = y[20000:]

split_at = len(train_x) - len(train_x) // 10
(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

Training Data:
(18000, 7, 12)
(18000, 4, 12)
Validation Data:
(2000, 7, 12)
(2000, 4, 12)
Testing Data:
(60000, 7, 12)
(60000, 4, 12)


In [ ]:
print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

# Build Model

In [11]:
print('Build model...')
model = Sequential()
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
_________________________________________________________________
activation_1 (Activation)    (None, 4, 12)             0         
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


# Training

In [12]:
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 0
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 3s 140us/step - loss: 2.0201 - acc: 0.2881 - val_loss: 1.8753 - val_acc: 0.3214
Q 379+396 T 775  ☒ 110 
Q 67+588  T 655  ☒ 110 
Q 72+334  T 406  ☒ 196 
Q 15+461  T 476  ☒ 19  
Q 831+35  T 866  ☒ 199 
Q 413+526 T 939  ☒ 113 
Q 88+31   T 119  ☒ 19  
Q 9+532   T 541  ☒ 19  
Q 49+330  T 379  ☒ 111 
Q 225+79  T 304  ☒ 189 

--------------------------------------------------
Iteration 1
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 97us/step - loss: 1.8467 - acc: 0.3294 - val_loss: 1.8175 - val_acc: 0.3235
Q 811+363 T 1174 ☒ 111 
Q 452+488 T 940  ☒ 131 
Q 386+580 T 966  ☒ 1113
Q 159+947 T 1106 ☒ 1111
Q 802+63  T 865  ☒ 133 
Q 672+67  T 739  ☒ 133 
Q 717+232 T 949  ☒ 111 
Q 714+673 T 1387 ☒ 111 
Q 633+71  T 704  ☒ 133 
Q 32+889  T 921  ☒ 133 

--------------------

18000/18000 [==============================] - 2s 92us/step - loss: 1.2343 - acc: 0.5421 - val_loss: 1.2415 - val_acc: 0.5316
Q 123+3   T 126  ☒ 222 
Q 20+650  T 670  ☒ 675 
Q 758+440 T 1198 ☒ 1100
Q 227+735 T 962  ☒ 991 
Q 9+279   T 288  ☒ 387 
Q 665+665 T 1330 ☒ 1322
Q 37+953  T 990  ☒ 997 
Q 34+677  T 711  ☒ 701 
Q 886+448 T 1334 ☒ 1312
Q 545+146 T 691  ☒ 676 

--------------------------------------------------
Iteration 15
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 90us/step - loss: 1.2035 - acc: 0.5531 - val_loss: 1.1863 - val_acc: 0.5598
Q 72+334  T 406  ☒ 410 
Q 488+2   T 490  ☒ 480 
Q 876+285 T 1161 ☒ 1114
Q 680+72  T 752  ☒ 788 
Q 80+170  T 250  ☒ 287 
Q 83+609  T 692  ☒ 688 
Q 703+57  T 760  ☒ 784 
Q 972+0   T 972  ☒ 920 
Q 18+47   T 65   ☒ 53  
Q 237+427 T 664  ☒ 682 

--------------------------------------------------
Iteration 16
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=======

18000/18000 [==============================] - 2s 90us/step - loss: 0.7814 - acc: 0.7123 - val_loss: 0.8131 - val_acc: 0.6889
Q 891+62  T 953  ☒ 955 
Q 309+974 T 1283 ☒ 1275
Q 49+703  T 752  ☒ 759 
Q 10+996  T 1006 ☒ 990 
Q 6+624   T 630  ☒ 629 
Q 21+888  T 909  ☒ 919 
Q 45+751  T 796  ☒ 797 
Q 684+371 T 1055 ☒ 1051
Q 74+924  T 998  ☒ 995 
Q 823+27  T 850  ☒ 859 

--------------------------------------------------
Iteration 29
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 90us/step - loss: 0.7322 - acc: 0.7290 - val_loss: 0.7388 - val_acc: 0.7190
Q 715+835 T 1550 ☒ 1557
Q 667+5   T 672  ☒ 671 
Q 811+25  T 836  ☒ 839 
Q 490+58  T 548  ☒ 549 
Q 14+734  T 748  ☒ 757 
Q 690+26  T 716  ☒ 717 
Q 68+320  T 388  ☑ 388 
Q 693+100 T 793  ☒ 884 
Q 886+448 T 1334 ☑ 1334
Q 580+284 T 864  ☒ 860 

--------------------------------------------------
Iteration 30
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=======

18000/18000 [==============================] - 2s 102us/step - loss: 0.2298 - acc: 0.9497 - val_loss: 0.2481 - val_acc: 0.9299
Q 346+63  T 409  ☑ 409 
Q 492+46  T 538  ☑ 538 
Q 83+1    T 84   ☒ 83  
Q 487+123 T 610  ☒ 500 
Q 425+88  T 513  ☑ 513 
Q 617+806 T 1423 ☑ 1423
Q 96+749  T 845  ☑ 845 
Q 507+344 T 851  ☑ 851 
Q 44+624  T 668  ☑ 668 
Q 201+759 T 960  ☑ 960 

--------------------------------------------------
Iteration 43
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 98us/step - loss: 0.2063 - acc: 0.9566 - val_loss: 0.2598 - val_acc: 0.9186
Q 303+38  T 341  ☑ 341 
Q 97+326  T 423  ☑ 423 
Q 7+199   T 206  ☑ 206 
Q 76+714  T 790  ☑ 790 
Q 511+13  T 524  ☒ 534 
Q 13+287  T 300  ☑ 300 
Q 572+76  T 648  ☑ 648 
Q 928+655 T 1583 ☑ 1583
Q 418+586 T 1004 ☒ 1014
Q 250+36  T 286  ☑ 286 

--------------------------------------------------
Iteration 44
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [======

18000/18000 [==============================] - 2s 88us/step - loss: 0.0711 - acc: 0.9910 - val_loss: 0.1066 - val_acc: 0.9699
Q 836+536 T 1372 ☑ 1372
Q 830+314 T 1144 ☑ 1144
Q 24+24   T 48   ☑ 48  
Q 848+3   T 851  ☑ 851 
Q 386+315 T 701  ☒ 601 
Q 131+17  T 148  ☒ 138 
Q 643+0   T 643  ☑ 643 
Q 367+68  T 435  ☑ 435 
Q 84+48   T 132  ☑ 132 
Q 41+10   T 51   ☒ 41  

--------------------------------------------------
Iteration 57
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 91us/step - loss: 0.1174 - acc: 0.9697 - val_loss: 0.1351 - val_acc: 0.9571
Q 349+250 T 599  ☑ 599 
Q 87+56   T 143  ☑ 143 
Q 261+268 T 529  ☒ 539 
Q 295+432 T 727  ☒ 737 
Q 21+13   T 34   ☒ 35  
Q 193+826 T 1019 ☑ 1019
Q 500+66  T 566  ☑ 566 
Q 130+326 T 456  ☑ 456 
Q 905+16  T 921  ☑ 921 
Q 416+731 T 1147 ☑ 1147

--------------------------------------------------
Iteration 58
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=======

18000/18000 [==============================] - 2s 87us/step - loss: 0.0279 - acc: 0.9972 - val_loss: 0.0719 - val_acc: 0.9774
Q 85+659  T 744  ☑ 744 
Q 745+416 T 1161 ☑ 1161
Q 833+5   T 838  ☑ 838 
Q 0+736   T 736  ☑ 736 
Q 69+933  T 1002 ☑ 1002
Q 4+383   T 387  ☑ 387 
Q 133+827 T 960  ☒ 950 
Q 554+22  T 576  ☑ 576 
Q 5+978   T 983  ☑ 983 
Q 503+640 T 1143 ☑ 1143

--------------------------------------------------
Iteration 71
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 87us/step - loss: 0.0266 - acc: 0.9977 - val_loss: 0.0595 - val_acc: 0.9838
Q 707+225 T 932  ☑ 932 
Q 65+86   T 151  ☑ 151 
Q 659+31  T 690  ☑ 690 
Q 879+40  T 919  ☑ 919 
Q 139+198 T 337  ☑ 337 
Q 550+562 T 1112 ☑ 1112
Q 818+80  T 898  ☑ 898 
Q 186+26  T 212  ☑ 212 
Q 65+419  T 484  ☑ 484 
Q 52+163  T 215  ☑ 215 

--------------------------------------------------
Iteration 72
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=======

18000/18000 [==============================] - 2s 97us/step - loss: 0.0985 - acc: 0.9663 - val_loss: 0.0856 - val_acc: 0.9722
Q 7+617   T 624  ☑ 624 
Q 956+45  T 1001 ☑ 1001
Q 80+754  T 834  ☑ 834 
Q 39+593  T 632  ☑ 632 
Q 48+806  T 854  ☑ 854 
Q 811+97  T 908  ☑ 908 
Q 632+134 T 766  ☑ 766 
Q 92+208  T 300  ☑ 300 
Q 5+955   T 960  ☑ 960 
Q 34+852  T 886  ☑ 886 

--------------------------------------------------
Iteration 85
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 95us/step - loss: 0.0256 - acc: 0.9954 - val_loss: 0.0484 - val_acc: 0.9864
Q 76+435  T 511  ☑ 511 
Q 358+495 T 853  ☑ 853 
Q 139+966 T 1105 ☑ 1105
Q 643+578 T 1221 ☑ 1221
Q 555+621 T 1176 ☑ 1176
Q 199+12  T 211  ☑ 211 
Q 883+55  T 938  ☑ 938 
Q 81+58   T 139  ☒ 149 
Q 604+220 T 824  ☑ 824 
Q 56+69   T 125  ☑ 125 

--------------------------------------------------
Iteration 86
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [=======

18000/18000 [==============================] - 2s 89us/step - loss: 0.0080 - acc: 0.9998 - val_loss: 0.0326 - val_acc: 0.9903
Q 349+250 T 599  ☑ 599 
Q 8+71    T 79   ☑ 79  
Q 4+504   T 508  ☑ 508 
Q 152+42  T 194  ☑ 194 
Q 906+80  T 986  ☑ 986 
Q 0+535   T 535  ☑ 535 
Q 755+90  T 845  ☑ 845 
Q 811+25  T 836  ☑ 836 
Q 559+647 T 1206 ☑ 1206
Q 382+870 T 1252 ☑ 1252

--------------------------------------------------
Iteration 99
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 2s 95us/step - loss: 0.0070 - acc: 0.9998 - val_loss: 0.0337 - val_acc: 0.9901
Q 15+115  T 130  ☑ 130 
Q 22+829  T 851  ☑ 851 
Q 820+817 T 1637 ☑ 1637
Q 996+118 T 1114 ☑ 1114
Q 413+526 T 939  ☑ 939 
Q 40+103  T 143  ☑ 143 
Q 902+78  T 980  ☑ 980 
Q 758+440 T 1198 ☑ 1198
Q 839+517 T 1356 ☑ 1356
Q 605+75  T 680  ☑ 680 


In [ ]:
print("MSG : Prediction")
test_x = ["555+275", "860+7  ", "340+29 "]
test_y = ["830 ", "867 ", "369 "]
x = np.zeros((len(test_x), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(test_y), DIGITS + 1, len(chars)), dtype=np.bool)
for j, (i, c) in enumerate(zip(test_x, test_y)):
    x[j] = ctable.encode(i, MAXLEN)
    y[j] = ctable.encode(c, DIGITS + 1)

# Validation

In [14]:
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))

Q 935+933 T 1868 ☑ 1868
Q 84+359  T 443  ☑ 443 
Q 28+107  T 135  ☑ 135 
Q 66+199  T 265  ☑ 265 
Q 343+110 T 453  ☑ 453 
Q 81+879  T 960  ☑ 960 
Q 228+7   T 235  ☑ 235 
Q 313+76  T 389  ☑ 389 
Q 618+66  T 684  ☑ 684 
Q 625+68  T 693  ☑ 693 
Q 816+967 T 1783 ☑ 1783
Q 557+800 T 1357 ☑ 1357
Q 963+886 T 1849 ☑ 1849
Q 567+0   T 567  ☑ 567 
Q 740+642 T 1382 ☑ 1382
Q 328+841 T 1169 ☑ 1169
Q 2+546   T 548  ☑ 548 
Q 35+742  T 777  ☑ 777 
Q 772+352 T 1124 ☑ 1124
Q 79+544  T 623  ☑ 623 
Q 367+89  T 456  ☑ 456 
Q 33+385  T 418  ☑ 418 
Q 101+521 T 622  ☒ 623 
Q 39+949  T 988  ☑ 988 
Q 918+450 T 1368 ☑ 1368
Q 706+31  T 737  ☑ 737 
Q 67+55   T 122  ☑ 122 
Q 332+73  T 405  ☑ 405 
Q 677+576 T 1253 ☑ 1253
Q 122+129 T 251  ☑ 251 
Q 0+647   T 647  ☑ 647 
Q 60+540  T 600  ☑ 600 
Q 129+33  T 162  ☑ 162 
Q 193+13  T 206  ☑ 206 
Q 27+893  T 920  ☑ 920 
Q 720+369 T 1089 ☑ 1089
Q 838+78  T 916  ☑ 916 
Q 457+49  T 506  ☑ 506 
Q 873+81  T 954  ☑ 954 
Q 712+490 T 1202 ☑ 1202
Q 63+173  T 236  ☑ 236 
Q 554+823 T 1377

Q 241+288 T 529  ☑ 529 
Q 8+726   T 734  ☑ 734 
Q 5+195   T 200  ☑ 200 
Q 689+744 T 1433 ☑ 1433
Q 855+898 T 1753 ☑ 1753
Q 85+711  T 796  ☑ 796 
Q 18+752  T 770  ☑ 770 
Q 64+214  T 278  ☑ 278 
Q 865+92  T 957  ☑ 957 
Q 20+365  T 385  ☑ 385 
Q 295+69  T 364  ☑ 364 
Q 491+495 T 986  ☒ 987 
Q 8+90    T 98   ☒ 97  
Q 77+713  T 790  ☑ 790 
Q 107+48  T 155  ☑ 155 
Q 563+38  T 601  ☑ 601 
Q 480+83  T 563  ☑ 563 
Q 916+8   T 924  ☑ 924 
Q 361+664 T 1025 ☑ 1025
Q 903+7   T 910  ☑ 910 
Q 653+18  T 671  ☑ 671 
Q 47+671  T 718  ☑ 718 
Q 86+461  T 547  ☑ 547 
Q 3+131   T 134  ☑ 134 
Q 38+161  T 199  ☒ 299 
Q 91+44   T 135  ☑ 135 
Q 751+272 T 1023 ☑ 1023
Q 653+679 T 1332 ☑ 1332
Q 349+863 T 1212 ☑ 1212
Q 137+21  T 158  ☑ 158 
Q 962+46  T 1008 ☑ 1008
Q 518+320 T 838  ☑ 838 
Q 830+933 T 1763 ☑ 1763
Q 86+432  T 518  ☑ 518 
Q 160+79  T 239  ☑ 239 
Q 87+231  T 318  ☑ 318 
Q 78+765  T 843  ☑ 843 
Q 668+107 T 775  ☑ 775 
Q 98+304  T 402  ☑ 402 
Q 757+10  T 767  ☑ 767 
Q 561+38  T 599  ☑ 599 
Q 81+280  T 361 

Q 98+449  T 547  ☑ 547 
Q 3+259   T 262  ☑ 262 
Q 99+964  T 1063 ☑ 1063
Q 722+536 T 1258 ☑ 1258
Q 186+70  T 256  ☑ 256 
Q 26+733  T 759  ☑ 759 
Q 155+949 T 1104 ☑ 1104
Q 553+3   T 556  ☑ 556 
Q 64+187  T 251  ☑ 251 
Q 542+118 T 660  ☑ 660 
Q 25+729  T 754  ☑ 754 
Q 928+84  T 1012 ☑ 1012
Q 351+973 T 1324 ☑ 1324
Q 94+87   T 181  ☑ 181 
Q 477+0   T 477  ☑ 477 
Q 153+39  T 192  ☑ 192 
Q 590+0   T 590  ☑ 590 
Q 74+561  T 635  ☑ 635 
Q 73+220  T 293  ☑ 293 
Q 63+226  T 289  ☑ 289 
Q 60+475  T 535  ☑ 535 
Q 882+873 T 1755 ☑ 1755
Q 598+3   T 601  ☑ 601 
Q 42+453  T 495  ☑ 495 
Q 364+834 T 1198 ☑ 1198
Q 64+177  T 241  ☑ 241 
Q 234+18  T 252  ☑ 252 
Q 534+882 T 1416 ☑ 1416
Q 569+851 T 1420 ☑ 1420
Q 219+67  T 286  ☑ 286 
Q 147+79  T 226  ☑ 226 
Q 783+181 T 964  ☑ 964 
Q 93+86   T 179  ☑ 179 
Q 457+360 T 817  ☑ 817 
Q 171+17  T 188  ☑ 188 
Q 31+126  T 157  ☑ 157 
Q 32+407  T 439  ☑ 439 
Q 17+789  T 806  ☑ 806 
Q 13+483  T 496  ☑ 496 
Q 402+350 T 752  ☑ 752 
Q 65+901  T 966  ☑ 966 
Q 89+487  T 576 

Q 68+103  T 171  ☑ 171 
Q 53+67   T 120  ☑ 120 
Q 450+56  T 506  ☑ 506 
Q 368+7   T 375  ☑ 375 
Q 49+471  T 520  ☑ 520 
Q 68+611  T 679  ☑ 679 
Q 888+97  T 985  ☑ 985 
Q 69+381  T 450  ☑ 450 
Q 580+39  T 619  ☑ 619 
Q 281+536 T 817  ☑ 817 
Q 958+94  T 1052 ☑ 1052
Q 3+250   T 253  ☑ 253 
Q 93+732  T 825  ☑ 825 
Q 81+646  T 727  ☑ 727 
Q 388+58  T 446  ☑ 446 
Q 20+503  T 523  ☑ 523 
Q 188+69  T 257  ☑ 257 
Q 212+839 T 1051 ☑ 1051
Q 153+8   T 161  ☑ 161 
Q 240+89  T 329  ☑ 329 
Q 206+704 T 910  ☒ 900 
Q 784+145 T 929  ☒ 939 
Q 792+99  T 891  ☑ 891 
Q 324+342 T 666  ☑ 666 
Q 56+437  T 493  ☑ 493 
Q 162+806 T 968  ☑ 968 
Q 919+446 T 1365 ☑ 1365
Q 495+69  T 564  ☑ 564 
Q 49+179  T 228  ☑ 228 
Q 169+572 T 741  ☑ 741 
Q 41+862  T 903  ☑ 903 
Q 19+565  T 584  ☑ 584 
Q 54+478  T 532  ☑ 532 
Q 51+995  T 1046 ☑ 1046
Q 344+90  T 434  ☑ 434 
Q 638+256 T 894  ☑ 894 
Q 93+82   T 175  ☑ 175 
Q 211+99  T 310  ☑ 310 
Q 630+253 T 883  ☑ 883 
Q 771+33  T 804  ☑ 804 
Q 603+702 T 1305 ☑ 1305
Q 74+499  T 573 

Q 573+15  T 588  ☑ 588 
Q 356+398 T 754  ☑ 754 
Q 622+430 T 1052 ☑ 1052
Q 84+166  T 250  ☑ 250 
Q 72+701  T 773  ☑ 773 
Q 442+52  T 494  ☑ 494 
Q 359+99  T 458  ☑ 458 
Q 77+328  T 405  ☑ 405 
Q 48+90   T 138  ☑ 138 
Q 534+179 T 713  ☑ 713 
Q 113+79  T 192  ☑ 192 
Q 278+205 T 483  ☑ 483 
Q 106+5   T 111  ☑ 111 
Q 65+844  T 909  ☑ 909 
Q 10+203  T 213  ☑ 213 
Q 356+406 T 762  ☑ 762 
Q 258+52  T 310  ☑ 310 
Q 56+485  T 541  ☑ 541 
Q 708+25  T 733  ☑ 733 
Q 386+826 T 1212 ☑ 1212
Q 572+814 T 1386 ☑ 1386
Q 871+16  T 887  ☑ 887 
Q 4+659   T 663  ☑ 663 
Q 257+59  T 316  ☑ 316 
Q 845+1   T 846  ☑ 846 
Q 440+24  T 464  ☑ 464 
Q 574+78  T 652  ☑ 652 
Q 62+999  T 1061 ☑ 1061
Q 137+17  T 154  ☑ 154 
Q 554+4   T 558  ☑ 558 
Q 904+51  T 955  ☑ 955 
Q 95+226  T 321  ☑ 321 
Q 552+227 T 779  ☑ 779 
Q 295+4   T 299  ☒ 390 
Q 740+3   T 743  ☑ 743 
Q 598+4   T 602  ☑ 602 
Q 73+609  T 682  ☑ 682 
Q 160+807 T 967  ☑ 967 
Q 228+303 T 531  ☑ 531 
Q 46+882  T 928  ☑ 928 
Q 424+455 T 879  ☑ 879 
Q 235+36  T 271 

...

Q 31+964  T 995  ☑ 995 
Q 909+839 T 1748 ☒ 1758
Q 189+46  T 235  ☑ 235 
Q 35+124  T 159  ☒ 169 
Q 999+773 T 1772 ☑ 1772
Q 3+336   T 339  ☑ 339 
Q 99+85   T 184  ☑ 184 
Q 3+504   T 507  ☑ 507 
Q 137+432 T 569  ☒ 579 
Q 5+738   T 743  ☑ 743 
Q 96+84   T 180  ☑ 180 
Q 95+663  T 758  ☑ 758 
Q 940+31  T 971  ☑ 971 
Q 1+999   T 1000 ☑ 1000
Q 75+473  T 548  ☑ 548 
Q 860+33  T 893  ☑ 893 
Q 29+963  T 992  ☑ 992 
Q 423+880 T 1303 ☑ 1303
Q 11+156  T 167  ☑ 167 
Q 636+80  T 716  ☑ 716 
Q 54+731  T 785  ☑ 785 
Q 933+66  T 999  ☒ 109 
Q 583+872 T 1455 ☑ 1455
Q 425+373 T 798  ☑ 798 
Q 993+15  T 1008 ☑ 1008
Q 594+371 T 965  ☑ 965 
Q 25+177  T 202  ☑ 202 
Q 992+116 T 1108 ☑ 1108
Q 557+44  T 601  ☑ 601 
Q 9+779   T 788  ☒ 798 
Q 347+5   T 352  ☑ 352 
Q 250+990 T 1240 ☑ 1240
Q 96+792  T 888  ☑ 888 
Q 50+441  T 491  ☑ 491 
Q 11+648  T 659  ☑ 659 
Q 19+42   T 61   ☒ 51  
Q 31+647  T 678  ☑ 678 
Q 26+421  T 447  ☑ 447 
Q 73+160  T 233  ☑ 233 
Q 257+447 T 704  ☒ 604 
Q 609+937 T 1546 ☑ 1546
Q 44+865  T 909 

Q 794+3   T 797  ☑ 797 
Q 101+581 T 682  ☑ 682 
Q 95+251  T 346  ☑ 346 
Q 663+762 T 1425 ☑ 1425
Q 438+17  T 455  ☑ 455 
Q 740+791 T 1531 ☑ 1531
Q 29+657  T 686  ☑ 686 
Q 587+59  T 646  ☑ 646 
Q 111+61  T 172  ☑ 172 
Q 22+291  T 313  ☑ 313 
Q 34+81   T 115  ☑ 115 
Q 743+405 T 1148 ☑ 1148
Q 6+912   T 918  ☑ 918 
Q 49+590  T 639  ☑ 639 
Q 407+44  T 451  ☑ 451 
Q 7+696   T 703  ☑ 703 
Q 613+191 T 804  ☑ 804 
Q 611+278 T 889  ☑ 889 
Q 12+540  T 552  ☑ 552 
Q 45+503  T 548  ☑ 548 
Q 253+5   T 258  ☑ 258 
Q 226+406 T 632  ☑ 632 
Q 689+448 T 1137 ☑ 1137
Q 748+336 T 1084 ☑ 1084
Q 994+91  T 1085 ☑ 1085
Q 388+70  T 458  ☑ 458 
Q 356+480 T 836  ☑ 836 
Q 534+276 T 810  ☑ 810 
Q 576+848 T 1424 ☑ 1424
Q 98+416  T 514  ☑ 514 
Q 64+689  T 753  ☑ 753 
Q 422+43  T 465  ☑ 465 
Q 376+23  T 399  ☒ 499 
Q 159+31  T 190  ☑ 190 
Q 134+792 T 926  ☑ 926 
Q 99+435  T 534  ☑ 534 
Q 881+33  T 914  ☑ 914 
Q 250+43  T 293  ☑ 293 
Q 15+62   T 77   ☑ 77  
Q 994+71  T 1065 ☑ 1065
Q 465+70  T 535  ☑ 535 
Q 542+24  T 566 

Q 925+248 T 1173 ☑ 1173
Q 83+309  T 392  ☒ 492 
Q 71+157  T 228  ☑ 228 
Q 215+23  T 238  ☑ 238 
Q 730+15  T 745  ☑ 745 
Q 898+95  T 993  ☑ 993 
Q 39+809  T 848  ☑ 848 
Q 67+564  T 631  ☑ 631 
Q 305+825 T 1130 ☑ 1130
Q 505+4   T 509  ☑ 509 
Q 56+380  T 436  ☑ 436 
Q 809+4   T 813  ☑ 813 
Q 4+309   T 313  ☑ 313 
Q 54+785  T 839  ☑ 839 
Q 169+360 T 529  ☑ 529 
Q 241+427 T 668  ☑ 668 
Q 42+62   T 104  ☑ 104 
Q 61+704  T 765  ☑ 765 
Q 37+160  T 197  ☑ 197 
Q 58+423  T 481  ☑ 481 
Q 838+733 T 1571 ☑ 1571
Q 27+167  T 194  ☑ 194 
Q 750+176 T 926  ☑ 926 
Q 15+852  T 867  ☑ 867 
Q 432+89  T 521  ☑ 521 
Q 810+63  T 873  ☑ 873 
Q 567+35  T 602  ☑ 602 
Q 12+279  T 291  ☑ 291 
Q 553+23  T 576  ☑ 576 
Q 774+16  T 790  ☑ 790 
Q 21+789  T 810  ☑ 810 
Q 799+22  T 821  ☑ 821 
Q 241+397 T 638  ☑ 638 
Q 49+563  T 612  ☑ 612 
Q 615+18  T 633  ☑ 633 
Q 150+552 T 702  ☑ 702 
Q 459+466 T 925  ☑ 925 
Q 794+0   T 794  ☑ 794 
Q 73+940  T 1013 ☑ 1013
Q 40+364  T 404  ☑ 404 
Q 389+77  T 466  ☑ 466 
Q 51+564  T 615 

Q 834+107 T 941  ☑ 941 
Q 150+736 T 886  ☑ 886 
Q 396+461 T 857  ☑ 857 
Q 330+86  T 416  ☑ 416 
Q 45+684  T 729  ☑ 729 
Q 64+299  T 363  ☑ 363 
Q 93+457  T 550  ☑ 550 
Q 452+234 T 686  ☑ 686 
Q 88+107  T 195  ☑ 195 
Q 493+30  T 523  ☑ 523 
Q 414+251 T 665  ☑ 665 
Q 772+14  T 786  ☑ 786 
Q 37+62   T 99   ☑ 99  
Q 776+633 T 1409 ☑ 1409
Q 16+957  T 973  ☑ 973 
Q 8+625   T 633  ☑ 633 
Q 501+739 T 1240 ☑ 1240
Q 253+7   T 260  ☑ 260 
Q 238+37  T 275  ☑ 275 
Q 240+151 T 391  ☒ 491 
Q 90+136  T 226  ☑ 226 
Q 575+38  T 613  ☑ 613 
Q 69+22   T 91   ☒ 90  
Q 874+2   T 876  ☑ 876 
Q 788+598 T 1386 ☑ 1386
Q 740+611 T 1351 ☑ 1351
Q 147+52  T 199  ☑ 199 
Q 107+253 T 360  ☑ 360 
Q 481+854 T 1335 ☑ 1335
Q 977+46  T 1023 ☑ 1023
Q 774+79  T 853  ☑ 853 
Q 577+239 T 816  ☑ 816 
Q 130+548 T 678  ☑ 678 
Q 941+907 T 1848 ☒ 1858
Q 556+23  T 579  ☑ 579 
Q 706+38  T 744  ☑ 744 
Q 91+452  T 543  ☑ 543 
Q 651+505 T 1156 ☑ 1156
Q 930+36  T 966  ☑ 966 
Q 761+25  T 786  ☑ 786 
Q 34+882  T 916  ☑ 916 
Q 995+17  T 1012

Q 865+761 T 1626 ☑ 1626
Q 767+75  T 842  ☑ 842 
Q 0+221   T 221  ☑ 221 
Q 67+150  T 217  ☑ 217 
Q 180+53  T 233  ☑ 233 
Q 25+87   T 112  ☑ 112 
Q 921+750 T 1671 ☑ 1671
Q 76+39   T 115  ☑ 115 
Q 912+665 T 1577 ☑ 1577
Q 69+77   T 146  ☑ 146 
Q 165+10  T 175  ☑ 175 
Q 954+485 T 1439 ☒ 1449
Q 859+415 T 1274 ☑ 1274
Q 20+917  T 937  ☑ 937 
Q 643+974 T 1617 ☑ 1617
Q 609+55  T 664  ☑ 664 
Q 17+87   T 104  ☑ 104 
Q 15+94   T 109  ☒ 119 
Q 315+63  T 378  ☑ 378 
Q 151+64  T 215  ☑ 215 
Q 74+969  T 1043 ☑ 1043
Q 993+7   T 1000 ☑ 1000
Q 23+682  T 705  ☑ 705 
Q 900+14  T 914  ☒ 913 
Q 80+981  T 1061 ☑ 1061
Q 743+507 T 1250 ☑ 1250
Q 597+474 T 1071 ☑ 1071
Q 682+527 T 1209 ☑ 1209
Q 212+176 T 388  ☑ 388 
Q 27+922  T 949  ☑ 949 
Q 50+115  T 165  ☑ 165 
Q 12+475  T 487  ☑ 487 
Q 31+305  T 336  ☑ 336 
Q 2+960   T 962  ☑ 962 
Q 66+692  T 758  ☑ 758 
Q 284+826 T 1110 ☑ 1110
Q 70+54   T 124  ☑ 124 
Q 548+22  T 570  ☑ 570 
Q 324+9   T 333  ☑ 333 
Q 144+603 T 747  ☑ 747 
Q 773+19  T 792  ☑ 792 
Q 651+895 T 1546